In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

all_data= pd.read_excel('output.xlsx')
# usecols='A:I'
#### Calculates the Gap distance
# all_data.loc[:,'Seq No':'Class'].head()
all_data['Date + Time'] = pd.to_datetime(all_data['Date + Time'])
all_data['Date'] = pd.to_datetime(all_data['Date + Time']).dt.date
all_data['Date'].unique()

all_data['Same Bound'] = all_data['Bound'] == all_data['Bound'].shift(1)
all_data['Same Lane'] = all_data['Lane'] == all_data['Lane'].shift(1)    
# if all_data.loc['Bound'].eq(all_data.loc['Bound'].shift(1)) and all_data.loc['Lane'].eq(all_data.loc['Lane'].shift(1)):
#     all_data['Gap Distance'] = all_data['Speed'] * (all_data['Date + Time'].shift(1) - all_data['Date + Time'])
# else:
#     all_data['Gap Distance'] = pd.NA


    
filter1 = all_data['Same Bound'] == True
filter2 = all_data['Same Lane'] == True
not_filter1 = all_data['Same Bound'] == False
not_filter2 = all_data['Same Lane'] == False

change = all_data['Date + Time'].diff().dt.seconds
# all_data['Speed'] * (all_data['Date + Time'].shift(1) - all_data['Date + Time'])

all_data.loc[filter1 & filter2, 'Gap Distance'] = change
all_data.loc[not_filter1, 'Gap Distance'] = pd.NA
all_data.loc[not_filter2, 'Gap Distance'] = pd.NA
# all_data = all_data.fillna(all_data['Gap Distance'].mean())

cols = ['Gap Distance']
all_data.loc[:,cols] = all_data.loc[:,cols].bfill()


/var/folders/py/yncddnwd6kn7d9dzfmz1kcbm0000gn/T/ipykernel_4301/54215659.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['Date + Time'] = pd.to_datetime(all_data['Date + Time'])


# Setup Functions

In [ ]:
# * Function Template for finding distribution from dataset, plotting the dataset, and interpolating data

from importlib import reload
import random
from collections import Counter

def finding_distribution_from_continuous(vehicle_class_key, vehicle_class_value , total_weight_key):
    # making a subset dataframe for the dataset that suit the selection
    tmp = all_data[all_data[vehicle_class_key] == vehicle_class_value]
    tmp = pd.DataFrame(tmp[total_weight_key].value_counts())

    tmp = tmp.sort_values(total_weight_key).reset_index()

    tmp.columns = [total_weight_key, 'Count']
    total_rows = tmp['Count'].sum()
    
    # calculate the percentage and cumulative probability for each dataset
    tmp['Percentage'] = tmp['Count'] / total_rows
    tmp['Cumulative Probability'] = tmp['Percentage'].cumsum()
    return tmp[[total_weight_key, 'Cumulative Probability']]

def plot_distribution(df:pd.DataFrame):
    print(df)
    import matplotlib.pyplot as plt 
    x_axis = df.columns[0]
    y_axis = df.columns[-1]
    plt.scatter(df[x_axis], df[y_axis], s=2)
    # plt.plot(df[x_axis], df[y_axis])
    plt=reload(plt)
    plt.title(f'{x_axis} against {y_axis}')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.grid()
    plt.show()
    return

def generate_and_interpolate_value(dataframe: pd.DataFrame):
    x_axis = dataframe.columns[0]
    y_axis = dataframe.columns[-1]

    # Generate a random number between 0 and 1
    random_prob = random.random()
    # print(f'{random_prob = }')

    # Sort the dataframe by cumulative probability column
    sorted_df = dataframe.sort_values(y_axis)

    # Find the closest value values based on the random probability
    closest_values = []
    closest_values.append(sorted_df[(sorted_df[y_axis] <= random_prob)].tail(1)[x_axis].values[0])
    closest_values.append(sorted_df[(sorted_df[y_axis] >= random_prob)].tail(1)[x_axis].values[0])
    # print(sorted_df[(sorted_df[y_axis] <= random_prob)].tail(1))
    # print(sorted_df[(sorted_df[y_axis] >= random_prob)].head(1))
    # print(closest_values)

    # Calculate the value value using linear interpolation
    interpolated_value = closest_values[0] + (closest_values[1] - closest_values[0]) * random_prob

    print(f'{random_prob = }, {interpolated_value = }')

    return interpolated_value

def generate_variable(vehicle_class_key: str, vehicle_class_value: int , total_weight_key: str)-> float :
    tmp = finding_distribution_from_continuous(vehicle_class_key, vehicle_class_value , total_weight_key)
    plot_distribution(tmp)
    output = generate_and_interpolate_value(tmp)
    return output

# Testing Usage:  
        # tmp = finding_distribution_from_continuous(vehicle_class_key, vehicle_class_value , total_weight_key)

# vehicle_class_key = 'Class'
# vehicle_class_value = 2
# total_weight_key = 'Total Weight'
# generate_variable(vehicle_class_key, vehicle_class_value , total_weight_key)

def axle_distribution_for_each_class() -> pd.DataFrame:
    X = []
    all_vehicle_classes = sorted(all_data['Class'].unique())
    diff_no_of_axles = [1,2,3,4,5,6,7]

    for vehicle_class in all_vehicle_classes:
        tmp = all_data[all_data['Class']== vehicle_class][['Total Weight','Gap Distance','No of Axle']]
        X.append(np.array(tmp['No of Axle']))

    df_no_of_axle = pd.DataFrame({'No of Axle':diff_no_of_axles})
    for i in range(len(X)):
        df_no_of_axle[all_vehicle_classes[i]] = Counter(X[i])
        df_no_of_axle[all_vehicle_classes[i]] = df_no_of_axle[all_vehicle_classes[i]] / df_no_of_axle[all_vehicle_classes[i]].sum()
    df_no_of_axle = df_no_of_axle.fillna(0)
    return df_no_of_axle
axle_distribution_for_each_class()

,No of Axle,1,2,3,4,5,6,7,8,9
0,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,1.0,0.977884,0.903141,0.977782,0.991215,0.222299,0.000000,0.427506,0.809524
3,4,0.0,0.022080,0.096859,0.017151,0.006955,0.538515,0.007448,0.572494,0.000000
4,5,0.0,0.000035,0.000000,0.005067,0.001830,0.239186,0.328777,0.000000,0.190476
5,6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.527953,0.000000,0.000000
6,7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.135822,0.000000,0.000000
